In [1]:
# raw data on April 1, 2022 from 16:10 to 16:40 UTC. Launch from SLC40 at 16:24 UTC, watched from Titusville
import os, sys, glob, obspy
import numpy as np
import pandas as pd
cwd = os.getcwd()
sys.path.append(os.path.join(cwd, 'campbell'))
import read_cs_files as campbell

HOME = os.getenv('HOME')
SDS_TOP = os.path.join(HOME, 'Desktop/Desktop - Glenn’s MacBook Air/KSC_new')
DROPBOX_TOP = os.path.join(HOME, 'Dropbox')
DROPBOX_PROJECT_DIR = os.path.join(DROPBOX_TOP, 'PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion')
EVENT_MSEED_DIR = os.path.join(DROPBOX_PROJECT_DIR, 'KSC_Well_Seismoacoustic_Data/SeismoAcousticData/Events')
HTML_DIR = './html'
PLOTS_DIR = HTML_DIR

WELLDATA_TOP = os.path.join(DROPBOX_PROJECT_DIR, 'KSC_Well_Seismoacoustic_Data/WellData')
TOB3_DIR = os.path.join(WELLDATA_TOP, 'Uploads')
WELLMSEED_DIR2 = os.path.join(WELLDATA_TOP, 'miniseed2')


def localtime2utc(this_dt):
    localTimeCorrection = 3600 * 4
    return this_dt + localTimeCorrection

# Read Campbell Scientific TOB3 file
allcolumns = []
tob3files = glob.glob(os.path.join(TOB3_DIR, '20220408/*.dat'))
lod = []
for tob3file in tob3files:
    data, meta = campbell.read_cs_files(tob3file, forcedatetime=False,
                      bycol=True, quiet=True, metaonly=False)
    
    

    df = pd.DataFrame(columns=meta[1])
    for c in range(len(meta[1])):
        df[meta[1][c]] = data[c]
        if not meta[1][c] in allcolumns:
            allcolumns.append(meta[1][c])
            
    print(allcolumns)
    print(meta)
    print(meta[1])
    print(df.columns)
    print(df)
    
    # convert to Miniseed
    local_startt = obspy.UTCDateTime(df.iloc[0]['TIMESTAMP'])
    nextt = obspy.UTCDateTime(df.iloc[1]['TIMESTAMP'])
    dt = nextt-local_startt
    utc_startt = localtime2utc(local_startt)

    tob3base = os.path.basename(tob3file)
    if tob3base[:3]=='HOF':
        sta = tob3base[9:11]
    elif tob3base[:8]=='Absolute':
        sta = 'AB'
    elif 'TOA5_HOF-IW000' in tob3base:
        sta = tob3base[-8:-6]
        
    print(sta)

    
    for this_col in df.columns[2:]:
        tr = obspy.Trace()
        tr.id = 'FL.S39%s.00.' % sta
        if this_col[1]=='_' and this_col[-1]=='A':
            tr.stats.station = 'AB6%s' % this_col[0] 
        if 'TempC' in this_col:
            tr.stats.channel = 'HKD'
        elif this_col=='Baro':
            tr.stats.channel = 'HDH'
        elif 'psi' in this_col:
            tr.stats.channel = 'HDD'
            if this_col[1]=='_' and this_col[-1]=='A':
                tr.stats.location = this_col[2:4] 
            else:
                tr.stats.location = this_col[:2]
        tr.stats.starttime = utc_startt
        tr.stats.delta = dt
        mseedfile = os.path.join(WELLMSEED_DIR2, '%s.%s' % (utc_startt.strftime('%Y%m%d%H%M%S'), tr.id  ) )
        tr.data = np.array(df[this_col])
        print('Writing %s' % mseedfile)
        tr.write(mseedfile, 'mseed')
        thisrow = {'TOB3file':tob3base, 'miniseedFile':os.path.basename(mseedfile)}
        lod.append(thisrow)
df = pd.DataFrame(lod)
#df.to_csv(os.path.join(WELLDATA_TOP,'lookuptable_tob3file_miniseedfile.csv'))

['TIMESTAMP', 'RECORD', 'PT(1)', 'PT(2)', 'PT(3)', 'PT(4)', 'CS109']
[['TOA5', 'HOF-IW0006I', 'CR6', '18084', 'CR6.Std.11.01', 'CPU:CR6 100hz.CR6', '15003', 'Test'], ['TIMESTAMP', 'RECORD', 'PT(1)', 'PT(2)', 'PT(3)', 'PT(4)', 'CS109'], ['TS', 'RN', '', '', '', '', ''], ['', '', 'Smp', 'Smp', 'Smp', 'Smp', 'Smp']]
['TIMESTAMP', 'RECORD', 'PT(1)', 'PT(2)', 'PT(3)', 'PT(4)', 'CS109']
Index(['TIMESTAMP', 'RECORD', 'PT(1)', 'PT(2)', 'PT(3)', 'PT(4)', 'CS109'], dtype='object')
                      TIMESTAMP    RECORD     PT(1)     PT(2)      PT(3)  \
0        2022-04-08 08:00:00.01  77115681  14.72682  1.079374   1.026047   
1        2022-04-08 08:00:00.02  77115682  14.72147  1.126209   1.012032   
2        2022-04-08 08:00:00.03  77115683  14.71961  1.122705   1.021801   
3        2022-04-08 08:00:00.04  77115684  14.72711  1.125685    1.07906   
4        2022-04-08 08:00:00.05  77115685  14.72764   1.09565  0.9984589   
...                         ...       ...       ...       ...       

ContextualVersionConflict: (idna 3.3 (/Users/thompsong/.local/lib/python3.8/site-packages), Requirement.parse('idna<3,>=2.5'), {'requests'})

In [12]:
len(data)

7

In [13]:
1439995/100/3600

3.9999861111111112